Единственная задача: обобщить написанный на семинаре код и сымитировать работу одного большого отдела ABBYY, в котором есть три маленьких подотдела с лингвистами, программистами и комплингом. То есть, что у нас должно быть реализовано:

- родительский класс "работник"
- базовые классы "лингвист", "программист" и "компьютерный лингвист"
- у всех методы work
- классы "босс_лингвист", "босс_программист" и "босс_кл", которые могут наследовать (с подмешиванием) от общего класса "босс"
- у боссов в атрибутах сидят их подчиненные
- босс подотдела получает квесты от менеджера главного отдела и принуждает сотрудников работать
- в главном отделе есть метод для выдачи квестов
- соответственно, используем как наследование, так и композицию с делегированием

In [82]:
from time import sleep
class Employee:
  def __init__(self, name, surname):
    self.name = name
    self.surname = surname
    self.__salary = 350  # _Employee__salary
    self.bankaccount = 0

  def work(self, *args):
    raise NotImplementedError

class Linguist(Employee):
  def __init__(self, name, surname):
    super().__init__(name, surname)
    self.read_papers = []
    self.__coef = 1.5 
    self.salary = self._Employee__salary * self._Linguist__coef
    self.publications = []
    
  def __repr__(self):
    return f"Linguist {self.name} {self.surname}"
    
  def work(self, task):
    if task.department == 'L':
      print('Working...')
      sleep(task.hours)
      self.bankaccount += self.salary * task.hours
      self.publications.append(task.title)
      print(f'{self.name} {self.surname} has published a paper {task.title} in Voprosy Yazykoznaniya')
    else:
      pass

  def readlingpapers(self, lingpaper):
    print('Reading...')
    sleep(len(lingpaper) // 10)
    self.read_papers.append(lingpaper)
    print(f'{self.name} {self.surname} has read {lingpaper}')

  def show_publications(self):
    print(f"Here is {self.name} {self.surname}'s publications list:")
    for p in self.publications:
      print(p)

class Programmer(Employee):
  def __init__(self, name, surname):
    super().__init__(name, surname)
    self.python_level = 0
    self.__coef = 2.5 
    self.salary = self._Employee__salary * self._Programmer__coef
    self.projects = []
  

  def work(self, task):
    if task.department == 'P':
      print('Working...')
      sleep(task.hours)
      self.bankaccount += self.salary * task.hours
      self.projects.append(task.title)
      print(f'{self.name} {self.surname} has committed a repo {task.title} to GitHub')
    else:
      pass

  def learnpython(self, hours):
    print('Studying...')
    sleep(hours)
    self.python_level += hours * 0.5
    print(f'{self.name} {self.surname} has learned a bit of Python')
  
  def show_projects(self):
    print(f"Here is {self.name} {self.surname}'s projects list:")
    for p in self.projects:
      print(p)



class ComputerLinguist(Linguist, Programmer):
  def __init__(self, name, surname):
    super().__init__(name, surname) #я подумала, что раз во всех классах, от которых мы наследуемся, есть имя и фамилия, то можно написать super, когда делала с base in bases призаведении экземпляра класса выпадала ошибка, мол, не хватает переданных аргументов (имени и фамилии)
    self.salary = self._Employee__salary * (self._Programmer__coef * 0.8)

  def work(self, task):
    if task.department == 'CL':
      print('Working...')
      sleep(task.hours)
      self.publications.append(task.title)
      self.bankaccount += self.salary * task.hours
      print(f'{self.name} {self.surname} has published {task.title} in ACL Papers')
    else:
      pass

  def show_publications(self):
    print(f"Here is {self.name} {self.surname}'s publications list:")
    for p in self.publications:
      print(p)

    



In [83]:
class Task:
    def __init__(self, hours : int, title, department='L'): #department = L/P/CL
        self.hours = hours
        self.title = title
        self.department = department
    def __repr__(self):
        return f"Task {self.title} takes {self.hours} to complete"

In [94]:
class Boss(Employee):
   '''По идее непосредственно боссов в процессе работы создаваться не должно, только боссы для конкретных отделов или менеджер'''
   def __init__(self, name, surname):
    super().__init__(name, surname)
    self.__coef = 1.7
    self.salary = self._Employee__salary * self._Boss__coef
    self.team = []
    self.tasks = []

   def add_to_team(self, *args):
      for w in args:
         self.team.append(w)

   def raise_qualification_all(self, func, *args):
      for worker in self.team:
         worker.func(*args)

   def get_a_task(self, task):
      self.tasks.append(task)
      print(f"{self.name} {self.surname} has been given a task {task.title}")


   def give_a_task(self, task, worker):
      for w in self.team:
         if w == worker:
            worker.work(task)
         else:
            continue
 
   def choose_a_worker(self):
      print('Your team:')
      for w in self.team: print(w.surname)
      worker_surname = input('Put in the surname of a worker to delegate the task')
      for w in self.team:
         if w.surname == worker_surname:
            return w
         else:
            print("Linguist wasn't found")
            return self.team[0]

   def delegate(self, task):
      self.give_a_task(task, self.choose_a_worker())


class Boss_ling(Boss, Linguist):
   def __init__(self, name, surname):
      for base in self.__class__.__bases__:
         base.__init__(self, name, surname)
      self.salary *= self._Linguist__coef

   def add_ling(self, ling=None): #можно и готового лингвиста добавить (но только если он правда лингвист) и нового родить
      if ling and ling.__class__.__name__ == 'Linguist':
         self.add_to_team(ling)
      else:
         name = input('Choose a name')
         surname = input('Choose a surname')
         ling = Linguist(name, surname)
         self.add_to_team(ling)

   def qualification_ling(self, lingpaper):
      self.raise_qualification_all(readlingpapers, lingpaper)

   def check_the_task(self, task):
      if task.department == 'L':
         return True
      else:
         return False


   

class Boss_prog(Boss, Programmer):
   def __init__(self, name, surname):
      for base in self.__class__.__bases__:
        base.__init__(self, name, surname)
      self.salary *= self._Programmer__coef

   def add_prog(self, prog=None):
      if prog and prog.__class__.__name__ == 'Programmer':
         self.add_to_team(prog)
      else:
         name = input('Choose a name')
         surname = input('Choose a surname')
         prog = Programmer(name, surname)
         self.add_to_team(prog)

   def check_the_task(self, task):
      if task.department == 'P':
         return True
      else:
         return False


   def qualification_prog(self, hours : int):
      self.raise_qualification_all(learnpython, hours)




class Boss_CL(Boss, ComputerLinguist):
   def __init__(self, name, surname):
      for base in self.__class__.__bases__:
        base.__init__(self, name, surname)
      self.salary *= (self._Programmer__coef * 0.8)
   
   def add_cl(self, cl=None):
      if cl and cl.__class__.__name__ == 'ComputerLinguist':
         self.add_to_team(cl)
      else:
         name = input('Choose a name')
         surname = input('Choose a surname')
         cl = ComputerLinguist(name, surname)
         self.add_to_team(cl)

   def check_the_task(self, task):
      if task.department == 'CL':
         return True
      else:
         return False


   def qualification_ling(self, arg):
      if type(arg) == int:
         self.raise_qualification_all(learnpython, arg)
      else:
         self.raise_qualification_all(readlingpapers, arg)

   

In [95]:
class HeadManager(Boss):
    def __init__(self, name, surname):
       super().__init__(name, surname)
       self.coef = 3
       self.salary *= self.coef
    
    def show_classes(self):
       c = []
       for worker in self.team:
          c.append(worker.__class__.__name__)
       return c


    def add_boss(self, boss=None):
      '''Есть возможность передать босса в функцию, но в процессе работы отдела используется создание боссов уже внутри функции'''
      if '__main__.Boss' in boss.__class__.__bases__ and boss.__class__.__name__ != 'HeadManager': #мы (я) не хотим, создавать менеджера в подчиненных у менеджера
         self.add_to_team(boss)
      else:
         name = input('Choose a name')
         surname = input('Choose a surname')
         department = input('Choose a department: ling, prog or cl')
         if department == 'ling' and not 'Boss_ling' in self.show_classes(): #второе условие должно проверять, чтобы в каждом отделе было по одному боссу
            boss = Boss_ling(name, surname)
         elif department == 'prog' and not 'Boss_prog' in self.show_classes():
            boss = Boss_prog(name, surname)
         elif not 'Boss_CL' in self.show_classes():
            boss = Boss_CL(name, surname)
         self.add_to_team(boss)

    def give_a_task(self, task):
      if task.department == 'L':
         for w in self.team:
            if w.__class__.__name__ == 'Boss_ling':
               w.get_a_task(task)
               return w
            else:
               continue
      elif task.department == 'P':
         for w in self.team:
            if w.__class__.__name__ == 'Boss_prog':
               w.get_a_task(task)
               return w
            else:
               continue
      else:
         for w in self.team:
            if w.__class__.__name__ == 'Boss_CL':
               w.get_a_task(task)
               return w
            else:
               continue
     
    def create_a_task(self):
       department = input('Choose a type of the task L (linguistics), P (programming) or CL (computational linguistics)')
       if department != 'L' and  department != 'P' and department != 'CL':
          department = 'CL'   
       title = input('Choose a title for the paper or project to write')
       hours = int(input('Choose how many hour the task is about to take (a number)')) 
       task = Task(hours, title, department=department)
       return task   
   
       

In [96]:
class Department:
    def __init__(self, name, head_manager_name, head_manager_surname):
        self.name = name
        self.departments = ['ling', 'prog', 'cl'] 
        self.head_manager = HeadManager(head_manager_name, head_manager_surname)
        print(f'The {self.name} Department has benn established. Head Manager {self.head_manager.name} {self.head_manager.surname} has been recruited.')
  
    def recruit_bosses(self):
        print("Recruiting bosses for the department...")
        for department in self.departments:
            print(f"Recruiting a boss for the {department} department.")
            self.head_manager.add_boss()

    def recruit_workers(self):
        for boss in self.head_manager.team:
            if isinstance(boss, Boss_ling): #это я под конец вспомнила, что можно проверять, является ли объект экземпляром класса не только через "obj.__class__.__name__"...
                amount = int(input('How many linguists do you want to recruite?'))
                print(f"Boss {boss.name} {boss.surname} is recruiting Linguists.")
                for x in range(amount):
                    boss.add_ling()
            elif isinstance(boss, Boss_prog):
                amount = int(input('How many programmer do you want to recruite?'))
                print(f"Boss {boss.name} {boss.surname} is recruiting Programmers.")
                for x in range(amount):
                    boss.add_prog()
            elif isinstance(boss, Boss_CL):
                amount = int(input('How many computer linguists do you want to recruite?'))
                print(f"Boss {boss.name} {boss.surname} is recruiting Computer Linguists.")
                for x in range(amount):
                    boss.add_cl()
                

    def raise_qualifications(self, department, *args): #повышаем квалификацию какому-нибуль отделу
         for boss in self.head_manager.team:
            if (department == 'L' and isinstance(boss, Boss_ling)):
                print("Raising qualifications for the Linguist department.")
                boss.qualification_ling(*args)  
            elif (department == 'P' and isinstance(boss, Boss_prog)):
                print("Raising qualifications for the Programmer department.")
                boss.qualification_prog(*args)  # Вызываем метод для программистов
            elif (department == 'CL' and isinstance(boss, Boss_CL)):
                print("Raising qualifications for the Computer Linguist department.")
                boss.qualification_cl(*args)  # Вызываем метод для компьютерных лингвистов
            else:
                print(f"Unknown department '{department}' or no appropriate boss found.")

    def show_structure(self):
        print("\nDepartment Structure:")
        print(f"Head Manager: {self.head_manager.name} {self.head_manager.surname}")
        for boss in self.head_manager.team:
            print(f"  Boss: {boss.name} {boss.surname} - {boss.__class__.__name__}")
            for worker in boss.team:
                print(f" Worker: {worker.name} {worker.surname} - {worker.__class__.__name__}")

    def start_work(self):
        self.recruit_bosses()
        self.recruit_workers()
        print("Work process has started!")
        while True:
            action = input("\nChoose an action: 'create a task', 'show the structure', 'raise qualification', or 'exit': ").lower()
            if action == 'create a task':
                task = self.head_manager.create_a_task()
                self.head_manager.give_a_task(task).delegate(task)
                print(f"Head Manager has delegated the '{task.title}' task to the appropriate boss.")

            elif action == 'show the structure':
                self.show_structure()
            elif action == 'raise qualification':
                department = input("Which department's qualifications to raise (L, P, or CL)? ")
                if department == 'L':
                    lingpaper = input("Enter the title of the paper for linguists to read: ")
                    self.raise_qualifications(department, lingpaper)
                elif department == 'P':
                    hours = int(input("How many hours should programmers study Python? "))
                    self.raise_qualifications(department, hours) 
                elif department == 'CL':
                    choice = input("Enter 'paper' for reading or 'python' for learning Python: ")
                    if choice == 'paper':
                        lingpaper = input("Enter the title of the paper for Computer Linguists to read: ")
                        self.raise_qualifications(department, lingpaper)  
                    else:
                        hours = int(input("How many hours should Computer Linguists study Python? "))
                        self.raise_qualifications(department, hours) 
                else:
                    print("Invalid department. Please enter 'L', 'P', or 'CL'.")
            elif action == 'exit':
                print("Ending the work process. Goodbye!")
                break
            else:
                print("Invalid action. Please choose again.")

In [97]:
d = Department('AB', 'F', 'd')
d.start_work()

The AB Department has benn established. Head Manager F d has been recruited.
Recruiting bosses for the department...
Recruiting a boss for the ling department.
Recruiting a boss for the prog department.
Recruiting a boss for the cl department.
Boss j n is recruiting Programmers.
Boss j cv is recruiting Computer Linguists.
Boss zs g is recruiting Linguists.
Work process has started!
zs g has been given a task sd
Your team:
v
Linguist wasn't found
Working...
vc  v has published a paper sd in Voprosy Yazykoznaniya
Head Manager has delegated the 'sd' task to the appropriate boss.
Ending the work process. Goodbye!


In [89]:
b = Boss('j', 'd')